# Обработка hashes и PE (ELF)-файлов на языке Python

Исходные файлы для блокнота находятся по [ссылке](https://github.com/dm-fedorov/infosec/tree/master/re-tools/samples).

Скачиваем весь архив с файлами для работы в Colab:

In [ ]:
!wget https://dfedorov.spb.ru/infosec/re/samples.zip

--2022-11-27 09:42:22--  https://dfedorov.spb.ru/infosec/re/samples.zip
Resolving dfedorov.spb.ru (dfedorov.spb.ru)... 92.53.96.244, 2a03:6f00:1::5c35:60f4
Connecting to dfedorov.spb.ru (dfedorov.spb.ru)|92.53.96.244|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11921 (12K) [application/zip]
Saving to: ‘samples.zip’

samples.zip         100%[===================>]  11.64K  --.-KB/s    in 0s      

2022-11-27 09:42:23 (128 MB/s) - ‘samples.zip’ saved [11921/11921]



In [ ]:
!unzip samples.zip

Archive:  samples.zip
   creating: samples/
 extracting: samples/v_01.txt        
  inflating: samples/main_03.c       
  inflating: samples/main_01.c       
  inflating: samples/task-1.exe      
  inflating: samples/test_01         
  inflating: samples/main_02.c       
  inflating: samples/all_hashes.txt  
  inflating: samples/test            
  inflating: samples/Makefile        
  inflating: samples/test_02         
  inflating: samples/test_03         


In [ ]:
!ls samples

all_hashes.txt	main_02.c  Makefile    test	test_02  v_01.txt
main_01.c	main_03.c  task-1.exe  test_01	test_03


## Определение сигнатуры файла

В системах GNU/Linux, чтобы найти сигнатуру файла (уникальная последовательность байтов), можно использовать команду [xxd](https://www.opennet.ru/man.shtml?topic=xxd&category=1&russian=0), которая генерирует шестнадцатеричный дамп файла, как показано ниже:

In [ ]:
!xxd samples/task-1.exe

00000000: 4d5a 9000 0300 0000 0400 0000 ffff 0000  MZ..............
00000010: b800 0000 0000 0000 4000 0000 0000 0000  ........@.......
00000020: 0000 0000 0000 0000 0000 0000 0000 0000  ................
00000030: 0000 0000 0000 0000 0000 0000 d000 0000  ................
00000040: 0e1f ba0e 00b4 09cd 21b8 014c cd21 5468  ........!..L.!Th
00000050: 6973 2070 726f 6772 616d 2063 616e 6e6f  is program canno
00000060: 7420 6265 2072 756e 2069 6e20 444f 5320  t be run in DOS 
00000070: 6d6f 6465 2e0d 0d0a 2400 0000 0000 0000  mode....$.......
00000080: b94b 04c7 fd2a 6a94 fd2a 6a94 fd2a 6a94  .K...*j..*j..*j.
00000090: a642 6b95 fe2a 6a94 fd2a 6b94 ff2a 6a94  .Bk..*j..*k..*j.
000000a0: fc47 6995 fc2a 6a94 fc47 6895 fc2a 6a94  .Gi..*j..Gh..*j.
000000b0: 5269 6368 fd2a 6a94 0000 0000 0000 0000  Rich.*j.........
000000c0: 0000 0000 0000 0000 0000 0000 0000 0000  ................
000000d0: 5045 0000 4c01 0400 bbe2 835d 0000 0000  PE..L......]....
000000e0: 0000 0000 e000 0201 0b01 0e16 0002 000

Видим, что исполняемые файлы ОС Windows, также называемые [PE-файлами](https://ru.wikipedia.org/wiki/Portable_Executable) (например, .exe, .dll, .com, .drv, .sys и т. д.), имеют подпись файла ```MZ``` или шестнадцатеричные символы ```4D 5A``` в первых двух байтах файла.

Выполним команду для [ELF-файла](https://ru.wikipedia.org/wiki/Executable_and_Linkable_Format) (подпись файла `ELF`): 

In [ ]:
!xxd samples/test_01

00000000: 7f45 4c46 0201 0100 0000 0000 0000 0000  .ELF............
00000010: 0300 3e00 0100 0000 8005 0000 0000 0000  ..>.............
00000020: 4000 0000 0000 0000 001a 0000 0000 0000  @...............
00000030: 0000 0000 4000 3800 0900 4000 1f00 1e00  ....@.8...@.....
00000040: 0600 0000 0500 0000 4000 0000 0000 0000  ........@.......
00000050: 4000 0000 0000 0000 4000 0000 0000 0000  @.......@.......
00000060: f801 0000 0000 0000 f801 0000 0000 0000  ................
00000070: 0800 0000 0000 0000 0300 0000 0400 0000  ................
00000080: 3802 0000 0000 0000 3802 0000 0000 0000  8.......8.......
00000090: 3802 0000 0000 0000 1c00 0000 0000 0000  8...............
000000a0: 1c00 0000 0000 0000 0100 0000 0000 0000  ................
000000b0: 0100 0000 0500 0000 0000 0000 0000 0000  ................
000000c0: 0000 0000 0000 0000 0000 0000 0000 0000  ................
000000d0: ac08 0000 0000 0000 ac08 0000 0000 0000  ................
000000e0: 0000 2000 0000 0000 0100 0000 0600 000

В следующем примере команда [file](https://www.opennet.ru/man.shtml?topic=file&category=1&russian=4) была запущена для двух разных файлов:

In [ ]:
!apt-get install file

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 5 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.4 [22.1 kB]
Fetched 275 kB in 1s (404 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 123991 files and d

In [ ]:
!file samples/task-1.exe

samples/task-1.exe: PE32 executable (GUI) Intel 80386, for MS Windows


In [ ]:
!file samples/test_01

samples/test_01: ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 2.6.32, BuildID[sha1]=f2bd5ec0510be9dee734eba2c607d4dab9694d56, not stripped


В Python модуль [python-magic](https://github.com/ahupp/python-magic) может использоваться для определения типа файла:

In [ ]:
!pip3 install python-magic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import magic

In [ ]:
magic.from_file("samples/test_01")

'ELF 64-bit LSB shared object, x86-64, version 1 (SYSV), dynamically linked, interpreter /lib64/ld-linux-x86-64.so.2, for GNU/Linux 2.6.32, BuildID[sha1]=f2bd5ec0510be9dee734eba2c607d4dab9694d56, not stripped'

In [ ]:
magic.from_file("samples/task-1.exe")

'PE32 executable (GUI) Intel 80386, for MS Windows'

## Обработка хеш-суммы на Python

В системе Linux хеш-суммы могут быть сгенерированы с использованием утилит [md5sum](https://www.opennet.ru/man.shtml?topic=md5sum&category=1&russian=0), [sha256sum](https://www.opennet.ru/man.shtml?topic=sha256sum&russian=0) и [sha1sum](https://www.opennet.ru/man.shtml?topic=sha1sum&russian=0):

In [ ]:
!md5sum samples/task-1.exe

a82a243ff5dbf90677c64eae4f0b6a8e  samples/task-1.exe


In [ ]:
!sha256sum samples/task-1.exe

c4b4e76d20cfb1159cd83a65c067fe56146e86ea11aa5c6228e20e5737e700b5  samples/task-1.exe


In [ ]:
!sha1sum samples/task-1.exe

79bcef7061fc9c79e4437871f8135498f8608b8f  samples/task-1.exe


В Python можно генерировать хеш-суммы, используя модуль [hashlib](https://docs.python.org/3/library/hashlib.html), как показано ниже:

In [ ]:
import hashlib
content = open("samples/task-1.exe","rb").read()
print(hashlib.md5(content).hexdigest())

a82a243ff5dbf90677c64eae4f0b6a8e


In [ ]:
print(hashlib.sha256(content).hexdigest())

c4b4e76d20cfb1159cd83a65c067fe56146e86ea11aa5c6228e20e5737e700b5


In [ ]:
print(hashlib.sha1(content).hexdigest())

79bcef7061fc9c79e4437871f8135498f8608b8f


## Извлечение строк

Извлечение строк может подсказать, как функционирует программа, и рассказать об индикаторах, указывающих на подозрительный двоичный код. Например, если вредоносная программа создает файл, имя файла сохраняется в виде строки в двоичном файле. Или если вредоносная программа разрешает доменное имя, контролируемое злоумышленником, это имя впоследствии хранится в виде строки. 

Чтобы извлечь строки из подозрительного двоичного файла, вы можете использовать утилиту [strings](https://www.opennet.ru/man.shtml?topic=strings) в системах GNU/Linux. 

Команда `strings` по умолчанию извлекает ASCII-строки, длина которых составляет минимум четыре символа. С помощью опции ```-a``` можно извлечь строки из целого файла. 

In [ ]:
!strings -a samples/task-1.exe

!This program cannot be run in DOS mode.
Rich
.text
`.rdata
@.data
.reloc
.text$mn
.idata$5
.rdata
.rdata$zzzdbg
.idata$2
.idata$3
.idata$4
.idata$6
.data
ExitProcess
WinExec
KERNEL32.dll
calc


В образцах вредоносных программ также используются Юникод-строки (2 байта на символ). Чтобы получить полезную информацию из двоичного файла, иногда нужно извлечь как ASCII-, так и Юникод-строки. Чтобы извлечь Юникод-строки с помощью команды `strings`, используйте опцию ```-el```:

In [ ]:
!strings -a -el samples/task-1.exe

Модуль [FLOSS](https://github.com/fireeye/flare-floss) автоматически извлекает запутанные строки из вредоносных программ.

Исполняемые файлы ОС Windows должны соответствовать формату PE/COFF (Portable Executable/Common Object File Format – Переносимый исполняемый/стандартный формат объектного файла).

Фактическое содержимое PE-файла разделено на секции. За ними сразу же следует PE-заголовок. Эти секции представляют либо код, либо данные, они имеют ```in-memory-атрибуты```, такие как чтение/запись. Секция, представляющая код, содержит инструкции, которые будут выполняться процессором, тогда как секция, содержащая данные, может представлять различные типы данных, такие как чтение/запись данных программы (глобальные переменные), таблицы импорта/экспорта, ресурсы и т. д. У каждой секции есть свое имя, которое передает ее назначение.

Например, секция с именем ```.text``` указывает на код и имеет атрибут ```read-execute```; раздел с именем ```.data``` указывает на глобальные данные и имеет атрибут ```read-write```.

Следующий скрипт Python демонстрирует использование модуля [pefile](https://github.com/erocarrera/pefile) для отображения секции и её характеристик:

In [ ]:
!pip3 install pefile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 989 kB/s 
  Created wheel for pefile: filename=pefile-2022.5.30-py3-none-any.whl size=69377 sha256=49afb64ee6932869d6a53ca4dc3844759e3c4f3b522ed7a379324464cca68e7f
  Stored in directory: /root/.cache/pip/wheels/93/de/9a/ff00177c0bd6f71ec50ece96f4994fc075b29f6ac1484f0bed
Successfully built pefile


In [ ]:
import pefile

pe = pefile.PE("samples/task-1.exe")
for section in pe.sections:
    print(f"{section.Name.decode()} \
    {hex(section.VirtualAddress)} \
    {hex(section.Misc_VirtualSize)} \
    {section.SizeOfRawData}")

.text        0x1000     0x1f     512
.rdata       0x2000     0x13a     512
.data        0x3000     0x5     512
.reloc       0x4000     0x10     512


Скрипт [Pescanner](https://github.com/hiddenillusion/AnalyzePE/blob/master/pescanner.py) использует эвристику вместо сигнатур и может помочь идентифицировать упакованные двоичные файлы, даже если для них нет сигнатур.